### 172
Make original month lag set

In [1]:
%load_ext autoreload
%autoreload 2
import gc
import re
import pandas as pd
from __future__ import print_function, division
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
import os
import sys
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import get_categorical_features, get_numeric_features, reduce_mem_usage
from preprocessing import get_dummies
import datetime

from tqdm import tqdm
import time
import sys

os.listdir('../input/')
key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_avtive_month']

### Data Load

In [2]:
df_train = utils.read_df_pkl('../input/train0*')
df_test = utils.read_df_pkl('../input/test0*')
df_train.set_index(key, inplace=True)
df_test.set_index(key, inplace=True)
# sys.exit()
df_hist = utils.read_df_pkl('../input/hist_clean_rdm0*')
df_hist = reduce_mem_usage(df_hist)
df_new = utils.read_df_pkl('../input/new_clean_rdm0*')
df_new = reduce_mem_usage(df_new)

  0%|          | 0/3 [00:00<?, ?it/s]

Mem. usage decreased to 2554.26 Mb (0.0% reduction)


100%|██████████| 3/3 [00:01<00:00,  2.22it/s]


Mem. usage decreased to 183.47 Mb (0.0% reduction)


In [3]:
df_trans = pd.concat([df_hist, df_new], axis=0)
train_test = pd.concat([df_train, df_test], axis=0)
print(train_test.shape)
print(df_trans.shape)

(325540, 5)
(31075392, 21)


### First Avtive Monthよりも前に購買履歴のあるcard_idをどのように扱うか?
original month_lagは、first_active_monthでなく、purchase_dateのminで作成する？  
いや、そうするとおかしなことになる。外れ値の半数は、first_active_monthから初回purchase_dateまでの期間が空いてるcard_idになっている。  
なので、purchase_dateを基準にoriginal month_lagをとると、初回購買まで期間が空いてる事実がなくなってしまう

In [4]:
# first active monthのNULL埋め
display(train_test[train_test['first_active_month'].isnull()])
train_test['first_active_month'].fillna('2017-03', inplace=True)
display(train_test[train_test['first_active_month'].isnull()])

# EDA first_active_monthよりmin purchase_dateが遅いcard_idの外れ値率は15%で、約半数になる。
# df_sp = df_train[(df_train['first_active_month'] != df_train['purchase_date'])]
# df_sp['target'].value_counts()
# df_late = df_sp[df_sp['first_active_month'] > df_sp['purchase_date']]
# df_late.head()

,feature_1,feature_2,feature_3,first_active_month,target
card_id,,,,,
C_ID_c27b4f80f7,5,2,1,NaN,NaN


,feature_1,feature_2,feature_3,first_active_month,target
card_id,,,,,


In [5]:
df_trans = df_trans[[key, 'purchase_date']]
df_trans['purchase_date'] = df_trans['purchase_date'].map(lambda x: str(x)[:7])
df_trans.drop_duplicates(inplace=True)
print(df_trans.shape)
df_trans.head()

(3044128, 2)


,card_id,purchase_date
0,C_ID_4e6213e9bc,2017-06
1,C_ID_4e6213e9bc,2017-07
2,C_ID_4e6213e9bc,2017-08
3,C_ID_4e6213e9bc,2017-09
4,C_ID_4e6213e9bc,2017-03


In [6]:
df_trans = df_trans.set_index(key).join(train_test['first_active_month'])
df_trans.head()

,purchase_date,first_active_month
card_id,,
C_ID_00007093c1,2017-10,2017-02
C_ID_00007093c1,2017-02,2017-02
C_ID_00007093c1,2017-05,2017-02
C_ID_00007093c1,2017-08,2017-02
C_ID_00007093c1,2017-07,2017-02


### days//30だと月ズレを起こすので、yyyymm同士の差分をとってmonth_lagをとる。基準0はfirst_active_month

In [11]:
lag_col = 'org_month_lag'
df_trans[lag_col] = df_trans.apply(lambda x: 
               ((int(str(x[0])[:4]) - int(str(x[1])[:4])) * 12) + (int(str(x[0])[-2:]) - int(str(x[1])[-2:]) )
               , axis=1
              )
df_trans.head()

,purchase_date,first_active_month,org_month_lag
card_id,,,
C_ID_00007093c1,2017-10,2017-02,8
C_ID_00007093c1,2017-02,2017-02,0
C_ID_00007093c1,2017-05,2017-02,3
C_ID_00007093c1,2017-08,2017-02,6
C_ID_00007093c1,2017-07,2017-02,5


In [12]:
df_trans['org_month_lag'].value_counts()

 4     218776
 3     218186
 2     212865
 0     210081
 1     204147
 5     201782
 6     182134
 7     163569
 8     145808
 9     130326
 10    116206
 11    104620
 12     94936
 13     86631
 14     77776
 15     69756
 16     61317
 17     53631
 18     48007
 19     43204
 20     39094
 21     34642
 22     30713
 23     27225
 24     24457
 25     22104
 26     20463
 27     18911
 28     17337
 29     15361
 30     14049
 31     12739
 32     11684
 33     10614
 34      9403
 35      8496
 36      7654
 37      6970
 38      6475
-1       6214
 39      5920
 40      5468
 41      4938
 42      4477
 43      4014
 44      3594
 45      3189
 46      2783
 47      2437
 48      2088
 49      1739
-2       1623
 50      1568
 51      1408
 52      1258
 53      1127
 54      1002
 55       853
-3        791
 56       760
 57       628
 58       509
-4        442
 59       428
 60       385
 61       342
 62       309
 63       264
-5        233
 64       215
-6        172
 65   

In [13]:
utils.to_pkl_gzip(obj=df_trans, path='../input/train_test_original_month_lag')